In [1]:
# !pip install -q transformers
# !pip install -q tf-models-official==2.2.0
# !pip install datasets

In [2]:
# !git clone https://github.com/dt024/VLSP2021_MRC.git

In [3]:
import os
import time
import datetime
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoConfig, AutoModel, BertModel, BertPreTrainedModel, BertConfig, RobertaConfig, RobertaModel, BertTokenizer, BertModel, RobertaTokenizer, XLMRobertaTokenizer
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup
import torch.nn as nn
from torch.autograd import Function
import torch.optim as optim
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from official import nlp
import official.nlp.optimization
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, WeightedRandomSampler
import logging
from tqdm import tqdm, trange


In [4]:
#Get the GPU device name
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
  print('GPU: {}'.format(device_name))
else:
  raise SystemError('GPU not found')


GPU: /device:GPU:0


In [5]:
# If there's a GPU
if torch.cuda.is_available():
  device = torch.device('cuda')
  torch.cuda.set_device(0)
  print('There are %d GPU(s).' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name())
else:
  print('No GPU, using CPU.')
  device = torch.device('cpu')


There are 1 GPU(s).
We will use the GPU: NVIDIA GeForce RTX 3090


In [6]:
base_dir    = 'datasets/'
train_path  = os.path.join(base_dir, 'fix-train.csv')
val_path    = os.path.join(base_dir, 'fix-val.csv')


In [7]:
df_train = pd.read_csv(train_path)
print(df_train.shape)
print(df_train.info())
display(df_train.head())


(25614, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25614 entries, 0 to 25613
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          25614 non-null  object
 1   context        25614 non-null  object
 2   answer_text    25614 non-null  object
 3   answer_start   25614 non-null  int64 
 4   id             25614 non-null  object
 5   is_impossible  25614 non-null  bool  
 6   question       25614 non-null  object
 7   answer_end     25614 non-null  int64 
dtypes: bool(1), int64(2), object(5)
memory usage: 1.4+ MB
None


,title,context,answer_text,answer_start,id,is_impossible,question,answer_end
0,Pháp,"Trong tháng 8 năm 1791, Hoàng đế Áo và Quốc vư...",nhóm 'Gironde' ủng hộ chiến tranh với Áo và Ph...,475,uit_024488,False,Hội nghị Lập pháp ở Pháp đã tồn tại hai phe đố...,568
1,Lịch sử Hoa Kỳ,Phe bảo hoàng mà người Anh trông cậy quá nhiều...,tháng 11 năm 1783,249,uit_017793,True,Đội quân cuối cùng của New Yord đã rời khỏi An...,266
2,Trung Hoa Dân Quốc (1912-1949),"Trong lĩnh vực toán học, Trung Hoa Dân Quốc đạ...",tiên phong và khai sáng vật lý học cận đại Tru...,177,uit_019463,False,Điều gì đã được làm bởi Ngô Hữu Huấn?,230
3,Iran,Iran là một thành viên sáng lập của Liên Hiệp ...,22 di sản,488,uit_022453,False,Có bao nhiêu di sản ở Iran được UNESCO công nhận?,497
4,Cách mạng Tháng Mười,"Ngày 10-1-1918, Đại hội Xô viết toàn Nga lần t...",Xô viết đại biểu nông dân với Xô viết đại biểu...,98,uit_011191,True,Đại hội đã quyết định thành lập hai Xô viết đạ...,167


In [8]:
df_val = pd.read_csv(val_path)
print(df_val.shape)
print(df_val.info())

display(df_val.head())


(2846, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2846 entries, 0 to 2845
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          2846 non-null   object
 1   context        2846 non-null   object
 2   answer_text    2846 non-null   object
 3   answer_start   2846 non-null   int64 
 4   id             2846 non-null   object
 5   is_impossible  2846 non-null   bool  
 6   question       2846 non-null   object
 7   answer_end     2846 non-null   int64 
dtypes: bool(1), int64(2), object(5)
memory usage: 158.5+ KB
None


,title,context,answer_text,answer_start,id,is_impossible,question,answer_end
0,Donald Trump,Một phần nhỏ tài sản của Trump nằm trong các k...,quyết định thâm nhập thị trường cổ phiếu,201,uit_015587,False,Hành động gì của Trump đã gây ra bất ngờ đối v...,241
1,Michael Jackson,Chuyến lưu diễn HIStory World Tour bắt đầu từ ...,"82 đêm nhạc tại 58 thành phố, đi qua 5 châu lụ...",152,uit_008735,False,Chặng đường của HIStory World Tour như thế nào?,250
2,Bức tường Berlin,Công dân Đông Đức đã được người dân Tây Berlin...,tặng 100 DM khi qua cổng,458,uit_021854,False,Chính phủ chào đón người dân Đông Đức như thế ...,482
3,Nhà Minh,"Sau khi Minh Thành Tổ lên ngôi, trong những nă...",triều đình nhận thấy tính quan trọng của mậu d...,1261,uit_018747,False,Vì sao triều đình lại từng bước xóa bỏ lệnh hả...,1344
4,Đế quốc La Mã,"Cuộc chiến tranh Do Thái-La Mã lần thứ nhất, đ...",Cả hai cuộc khởi nghĩa này đều bị đàn áp dã man,800,uit_020519,False,Kết quả của hai cuộc khởi nghĩa của người Do T...,847


In [9]:
train_q  = df_train.question.values
train_a  = df_train.answer_text.values
train_c  = df_train.context.values
train_title = df_train.title.values
train_id = df_train.id.values
train_impossible = df_train.is_impossible.values
train_ans_start = df_train.answer_start.values
train_ans_end = df_train.answer_end.values

val_q  = df_val.question.values
val_a  = df_val.answer_text.values
val_c  = df_val.context.values
val_id = df_val.id.values
val_title = df_val.title.values
val_impossible = df_val.is_impossible.values
val_ans_start = df_val.answer_start.values
val_ans_end = df_val.answer_end.values

train_data = [train_id, train_q, train_title, train_c, train_a, train_impossible, train_ans_start]
val_data = [val_id, val_q, val_title, val_c, val_a, val_impossible, val_ans_start]

In [10]:
dev_answers = []
for i in range(len(val_id)):
  temp2 = {}
  temp2['id'] = val_id[i]
  temp = {}
  temp['answer_start'] = [val_ans_start[i]]
  temp['text'] = [val_a[i]]
  temp2['answers'] = temp
  dev_answers.append(temp2)

In [11]:
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
# tokenizer = AutoTokenizer.from_pretrained(MODEL, do_lower_case=False)
tokenizer = XLMRobertaTokenizer.from_pretrained('ancs21/xlm-roberta-large-vi-qa', do_lower_case=False)


Loading BERT tokenizer...


In [12]:
print(tokenizer.tokenize(train_q[0]))
print(train_q[0])

['▁Hội', '▁nghị', '▁Lập', '▁pháp', '▁ở', '▁Pháp', '▁đã', '▁tồn', '▁tại', '▁hai', '▁', 'phe', '▁đối', '▁lập', '▁nào', '?']
Hội nghị Lập pháp ở Pháp đã tồn tại hai phe đối lập nào?


In [13]:
from utils import init_logger
from trainer import Trainer

In [14]:
class arg():
  def __init__(self):
    self.model_dir = '.'
    self.model_name_or_path = 'ancs21/xlm-roberta-large-vi-qa'
    self.do_train = True
    self.do_eval = True
    self.no_cuda = False
    # self.logging_steps = 13         #Log every X updates steps
    # self.save_steps = 13            #Save checkpoint every X updates steps
    self.warmup_steps = 0
    self.max_grad_norm = 1.0
    self.adam_epsilon = 1e-8
    self.weight_decay = 0.01
    self.dropout_rate = 0.2
    self.max_steps = -1              #total number of training steps to perform
    self.gradient_accumulation_steps = 1     #Number of updates steps to accumulate before performing a backward/update pass
    self.num_train_epochs = 2
    self.learning_rate = 2e-5
    self.early_stop = 2
    self.train_batch_size = 8
    self.eval_batch_size = 8
    self.seed = 2021
    self.max_seq_length = 450
    self.doc_stride = 150
    self.max_query_length = 150
    self.do_lower_case= False
    self.version_2_with_negative = True
    self.null_score_diff_threshold=0.0
    self.n_best_size = 100
    self.max_answer_length = 350

In [15]:
args = arg()

In [16]:
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if not args.no_cuda and torch.cuda.is_available():
        torch.cuda.manual_seed_all(args.seed)

init_logger()
set_seed(args)

In [17]:
logger = logging.getLogger(__name__)

trainer = Trainer(args, train_dataset=train_data, dev_dataset=val_data, dev_answer=dev_answers, tokenizer=tokenizer)

add example index and unique id: 100%|██████████| 2846/2846 [00:00<00:00, 1330026.65it/s]


In [18]:
trainer.train()

10/18/2021 15:47:48 - INFO - trainer -   ***** Running training *****
10/18/2021 15:47:48 - INFO - trainer -     Num examples = 26506
10/18/2021 15:47:48 - INFO - trainer -     Num Epochs = 2
10/18/2021 15:47:48 - INFO - trainer -     Total train batch size = 8
10/18/2021 15:47:48 - INFO - trainer -     Gradient Accumulation steps = 1
10/18/2021 15:47:48 - INFO - trainer -     Total optimization steps = 6626
10/18/2021 15:47:48 - INFO - trainer -     Logging steps = 3313
10/18/2021 15:47:48 - INFO - trainer -     Save steps = 3313
10/18/2021 15:47:49 - INFO - trainer -   Epoch: [1][0/3313] Elapsed 0m 0s (remain 37m 26s) Loss: 7.6791 
10/18/2021 15:48:31 - INFO - trainer -   Epoch: [1][100/3313] Elapsed 0m 42s (remain 22m 28s) Loss: 2.3156 
10/18/2021 15:49:13 - INFO - trainer -   Epoch: [1][200/3313] Elapsed 1m 24s (remain 21m 45s) Loss: 2.0474 
10/18/2021 15:49:55 - INFO - trainer -   Epoch: [1][300/3313] Elapsed 2m 6s (remain 21m 4s) Loss: 1.8963 
10/18/2021 15:50:37 - INFO - trainer

(6626, 2.2612778058013023)

In [24]:
pred = trainer.evaluate("dev",out_pred=True)

10/17/2021 07:32:55 - INFO - VLSP2021_MRC.trainer -   ***** Running evaluation on dev dataset *****
10/17/2021 07:32:55 - INFO - VLSP2021_MRC.trainer -     Num examples = 28
10/17/2021 07:32:55 - INFO - VLSP2021_MRC.trainer -     Batch size = 8
10/17/2021 07:32:57 - INFO - VLSP2021_MRC.trainer -   VAL: [4/4] Elapsed 0m 0s (remain 0m 0s) Loss: 5.2583 
10/17/2021 07:32:57 - INFO - VLSP2021_MRC.trainer -   ***** Eval results *****
10/17/2021 07:32:57 - INFO - VLSP2021_MRC.trainer -     exact_match = 0.0000
10/17/2021 07:32:57 - INFO - VLSP2021_MRC.trainer -     f1 = 7.7357
10/17/2021 07:32:57 - INFO - VLSP2021_MRC.trainer -     loss = 5.2583


In [25]:
pred

[OrderedDict([('text', 'suất thấp của tiền gửi ngân hàng.'),
              ('probability', 0.05170219820190293),
              ('start_logit', 0.6922128796577454),
              ('end_logit', 0.9497385025024414),
              ('start_index', 69),
              ('end_index', 75),
              ('qas_id', 'uit_015587')]),
 OrderedDict([('text',
               '1997; Rowe hạ sinh thêm bé gái Paris-Michael Katherine Jackson vào ngày 3 tháng 4 năm 1998. Cả hai ly hôn năm 1999'),
              ('probability', 0.06081818457403736),
              ('start_logit', 1.0934288501739502),
              ('end_logit', 0.9688441157341003),
              ('start_index', 209),
              ('end_index', 231),
              ('qas_id', 'uit_008735')]),
 OrderedDict([('text',
               'diễu hành, những người hoàn toàn xa lạ ôm choàng lấy nhau.'),
              ('probability', 0.05560178677764208),
              ('start_logit', 0.7965071201324463),
              ('end_logit', 1.103086233139038),
    

In [26]:
dev_answers

[{'answers': {'answer_start': [201],
   'text': ['quyết định thâm nhập thị trường cổ phiếu']},
  'id': 'uit_015587'},
 {'answers': {'answer_start': [152],
   'text': ['82 đêm nhạc tại 58 thành phố, đi qua 5 châu lục và 35 quốc gia, phục vụ hơn 4.5 triệu người hâm mộ']},
  'id': 'uit_008735'},
 {'answers': {'answer_start': [458], 'text': ['tặng 100 DM khi qua cổng']},
  'id': 'uit_021854'},
 {'answers': {'answer_start': [1261],
   'text': ['triều đình nhận thấy tính quan trọng của mậu dịch đối ngoại đối với cư dân ven biển']},
  'id': 'uit_018747'},
 {'answers': {'answer_start': [800],
   'text': ['Cả hai cuộc khởi nghĩa này đều bị đàn áp dã man']},
  'id': 'uit_020519'},
 {'answers': {'answer_start': [152],
   'text': ['sự cai trị hà khắc của người Đức tại các vùng lãnh thổ chiếm đóng']},
  'id': 'uit_009913'},
 {'answers': {'answer_start': [453],
   'text': ['Ronald Reagan được bầu làm tổng thống vào năm 1980']},
  'id': 'uit_018206'},
 {'answers': {'answer_start': [100],
   'text': [